In [96]:
import sys

from sem_covid import config
from sem_covid.services.store_registry import StoreRegistry

sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

In [47]:
from sem_covid.services.data_registry import Dataset

In [5]:

import tensorflow_hub as hub
import numpy as np

In [6]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print("module %s loaded" % module_url)


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [7]:
sentences = ["I ate dinner.",
             "We had a three-course meal.",
             "Brad came to dinner with us.",
             "He loves fish tacos.",
             "In the end, we all felt like we ate too much.",
             "We all agreed; it was a magnificent evening."]

In [8]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [17]:
sentence_embeddings = model(sentences)
query = "Sam came to breakfast with us."
query_vec = model([query])[0]

In [34]:
my_dict = {}
for index in range(0, len(sentences)):
    my_dict[sentences[index]] = list(sentence_embeddings[index])

In [29]:
import pandas as pd
from scipy.spatial.distance import euclidean, pdist, squareform


def similarity_func(u, v):
    return 1 / (1 + euclidean(u, v))

In [39]:
DF_var = pd.DataFrame.from_dict(
    {"s1": [1.2, 3.4, 10.2], "s2": [1.4, 3.1, 10.7], "s3": [2.1, 3.7, 11.3], "s4": [1.5, 3.2, 10.9]})
DF_var.index = ["g1", "g2", "g3"]

dists = pdist(DF_var, similarity_func)
DF_euclid = pd.DataFrame(squareform(dists), columns=DF_var.index, index=DF_var.index)

In [42]:
from sklearn.metrics.pairwise import pairwise_distances

In [46]:
pd.DataFrame(pairwise_distances(sentence_embeddings, metric=similarity_func), columns=sentences, index=sentences)

,I ate dinner.,We had a three-course meal.,Brad came to dinner with us.,He loves fish tacos.,"In the end, we all felt like we ate too much.",We all agreed; it was a magnificent evening.
I ate dinner.,1.000000,0.528034,0.473817,0.442395,0.451878,0.434089
We had a three-course meal.,0.528034,1.000000,0.475007,0.453022,0.468335,0.465753
Brad came to dinner with us.,0.473817,0.475007,1.000000,0.435233,0.424850,0.439880
He loves fish tacos.,0.442395,0.453022,0.435233,1.000000,0.428430,0.425791
"In the end, we all felt like we ate too much.",0.451878,0.468335,0.424850,0.428430,1.000000,0.476045
We all agreed; it was a magnificent evening.,0.434089,0.465753,0.439880,0.425791,0.476045,1.000000


In [35]:
my_df = pd.DataFrame(my_dict)

In [38]:
squareform(pdist(my_df, similarity_func))

array([[0.        , 0.90051244, 0.81759991, ..., 0.84775377, 0.83364561,
        0.91999471],
       [0.90051244, 0.        , 0.88150386, ..., 0.90325164, 0.88599647,
        0.85497471],
       [0.81759991, 0.88150386, 0.        , ..., 0.87786559, 0.89468757,
        0.78265109],
       ...,
       [0.84775377, 0.90325164, 0.87786559, ..., 0.        , 0.86845549,
        0.83475636],
       [0.83364561, 0.88599647, 0.89468757, ..., 0.86845549, 0.        ,
        0.80037572],
       [0.91999471, 0.85497471, 0.78265109, ..., 0.83475636, 0.80037572,
        0.        ]])

In [18]:
for sent in sentences:
    sim = cosine(query_vec, model([sent])[0])
    print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  I ate dinner. ; similarity =  0.2298401
Sentence =  We had a three-course meal. ; similarity =  0.3595812
Sentence =  Brad came to dinner with us. ; similarity =  0.44425395
Sentence =  He loves fish tacos. ; similarity =  0.15107124
Sentence =  In the end, we all felt like we ate too much. ; similarity =  0.13258228
Sentence =  We all agreed; it was a magnificent evening. ; similarity =  0.15228067


In [48]:
df = Dataset.PWDB.fetch()

100% (1288 of 1288) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [51]:
TEXTUAL_COLUMNS = ['title', 'background_info_description', 'content_of_measure_description',
                   'use_of_measure_description', 'involvement_of_social_partners_description']

In [53]:
df['text'] = df[TEXTUAL_COLUMNS].agg('. '.join, axis=1)

In [77]:
df['emb'] = model(df['text'].tolist()).numpy().tolist()

In [82]:
pairwise_distances(df['emb'].tolist(), metric=similarity_func)

array([[1.        , 0.49651195, 0.4537221 , ..., 0.47857887, 0.51591195,
        0.50818774],
       [0.49651195, 1.        , 0.45474067, ..., 0.48037731, 0.48482221,
        0.46447182],
       [0.4537221 , 0.45474067, 1.        , ..., 0.46535745, 0.44141898,
        0.4505972 ],
       ...,
       [0.47857887, 0.48037731, 0.46535745, ..., 1.        , 0.509062  ,
        0.46523969],
       [0.51591195, 0.48482221, 0.44141898, ..., 0.509062  , 1.        ,
        0.5196861 ],
       [0.50818774, 0.46447182, 0.4505972 , ..., 0.46523969, 0.5196861 ,
        1.        ]])

In [85]:
df['document_segment'] = 'all'

In [93]:
DOCUMENT_TEXT = 'text'
DOCUMENT_SEGMENT = 'segment_type'
DOCUMENT_SEGMENT_ID = 'segment_id'
DOCUMENT_SOURCE = 'source'
DOCUMENT_PREPARE_METHOD = 'prepare_method'
DOCUMENT_EMBEDDING = 'embedding'
DOCUMENT_TEXTUAL_COLUMNS ='textual_columns'
DOCUMENT_EMBEDDINGS_FEATURE_STORE_NAME = 'fs_document_embeddings'

In [133]:
class DocumentEmbeddingPipeline:

    def __init__(self, es_index_name: str, textual_columns: list):
        self.es_index_name = es_index_name
        self.textual_columns = textual_columns
        self.prepared_dataset = None
        self.dataset = None
        self.model = None

    def load_documents(self):
        es_store = StoreRegistry.es_index_store()
        self.dataset = es_store.get_dataframe(self.es_index_name)

    def prepare_textual_columns(self):
        assert self.dataset is not None
        self.dataset.dropna(subset = self.textual_columns, inplace=True)
        self.dataset[DOCUMENT_TEXT] = self.dataset[self.textual_columns].agg('. '.join, axis=1)
        self.prepared_dataset = pd.DataFrame(self.dataset[DOCUMENT_TEXT])
        self.prepared_dataset[DOCUMENT_SOURCE] = self.es_index_name
        self.prepared_dataset[DOCUMENT_SEGMENT] = 'document'
        self.prepared_dataset[DOCUMENT_SEGMENT_ID] = 0
        self.prepared_dataset[DOCUMENT_PREPARE_METHOD] = 'join_textual_columns'
        self.prepared_dataset[DOCUMENT_TEXTUAL_COLUMNS] = [self.textual_columns] * len(self.prepared_dataset)

    def load_model(self):
        model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
        self.model = hub.load(model_url)

    def compute_embeddings(self):
        assert self.model is not None
        assert self.prepared_dataset is not None
        self.prepared_dataset[DOCUMENT_EMBEDDING] = self.model(self.prepared_dataset[DOCUMENT_TEXT].tolist()).numpy().tolist()

    def store_embeddings(self):
        assert self.prepared_dataset is not None
        assert DOCUMENT_EMBEDDING in self.prepared_dataset.columns
        feature_store = StoreRegistry.es_feature_store()
        feature_store.put_features(features_name=DOCUMENT_EMBEDDINGS_FEATURE_STORE_NAME,
                                   content=self.prepared_dataset)

    def execute(self):
        self.load_documents()
        self.prepare_textual_columns()
        self.load_model()
        self.compute_embeddings()
        self.store_embeddings()

In [134]:
document_embedding_pipeline = DocumentEmbeddingPipeline(es_index_name=config.IRELAND_TIMELINE_ELASTIC_SEARCH_INDEX_NAME,
                                                        textual_columns=['title']
                                                        )

In [135]:
document_embedding_pipeline.execute()


100% (410 of 410) |######################| Elapsed Time: 0:00:00 Time:  0:00:00
N/A% (0 of 407) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

In [129]:
df = Dataset.IRELAND_ACTION_TIMELINE.fetch()

In [130]:
len(df)

410

In [131]:
df.dropna(subset = ["title"], inplace=True)

In [132]:
len(df)

407

In [83]:
sim_matrix = pd.DataFrame(pairwise_distances(df['emb'].tolist(), metric=similarity_func), columns=df['text'].tolist(),
                          index=df['text'].tolist())

In [84]:
sim_matrix

Agreement on a teleworking regime. During the COVID-19 crisis, teleworking has been identified as a vital pillar to keep companies working and prevent social hardship. Discussions between the two representative social partners OGBL and LCGB and the employer association UEL, together with the Ministry of Work and Employment, led in a first instance to a joint assessment on teleworking at the level of the Economic and Social Council CES. From there, discussions continued between the social partners and an inter-professional agreement could have been signed between the social partners in October 2020. . Applied for a period of three years and covering all sectors in Luxembourg (with the exception of transport), the agreement provides a definition of teleworking: \r\r* Teleworking is identified as a form of organisational work, conducted by digital means and usually done at the company, but that is transferred to the location where the employee lives. \r* The work is considered as teleworking when it is applied to occasional and exceptional circumstances, and when it remains below the 10% threshold of the annual working time. \r* Teleworking is based on a written agreement between the employer and the employee, containing several compulsory elements, such as for example the location where telework takes place and the number of hours, and an employee can't be dismissed if he/she does not accept a teleworking scheme and an employer can't be made accountable if the employer refuses a teleworking scheme. \r* The employer must also provide to the employee all the technical equipment to efficiently work under a teleworking scheme. \r* The company staff delegation is regularly informed about the teleworking schemes. . All employees in the private sector (with the exception of transport sector) and covered by the Code of Work are concerned.. The agreement is a social partner initiative within the Economic and Social Committee. The two representative trade unions OGBL and LCGB, as well as the employer association UEL signed the agreement.   \
Agreement on a teleworking regime. During the C...                                           1.000000                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [109]:
feature_store = StoreRegistry.es_feature_store()

In [138]:
df = feature_store.get_features(DOCUMENT_EMBEDDINGS_FEATURE_STORE_NAME)

100% (4519 of 4519) |####################| Elapsed Time: 0:00:01 Time:  0:00:01


In [148]:
len(df[DOCUMENT_EMBEDDING][0])

512

In [144]:
import faiss

In [204]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(len(df[DOCUMENT_EMBEDDING][0])))

In [205]:
index.add_with_ids(np.array(df[DOCUMENT_EMBEDDING].to_list()).astype('float32'),
                   np.array(range(0,len(df))))


In [238]:
query = df[DOCUMENT_EMBEDDING].to_list()[2]

In [200]:
df.iloc[3][['text','source']]

text      Waiver of advance payments for social and heal...
source                                              ds_pwdb
Name: 18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044, dtype: object

In [199]:
df.iloc[756][['text','source']]

text      Postponement of the employers' deadline for pa...
source                                              ds_pwdb
Name: 01a95e4b5c3c887bfd01a05c21b914df2248bd0b68da4785e7dec062bbedfa9e, dtype: object

In [239]:
query

[0.0378753208,
 -0.038546972,
 -0.0331166461,
 -0.0483590327,
 0.022393113,
 0.003940654,
 0.0649666637,
 -0.0591262542,
 -0.0589584373,
 -0.0070893406,
 -0.0574852228,
 0.0394048467,
 0.0607795827,
 -0.0592336655,
 0.0646905378,
 -0.0220377017,
 0.0651763603,
 -0.0132389721,
 -0.059833575,
 -0.0399091952,
 -0.0023563665,
 0.0053202608,
 -0.0181395654,
 0.025766328,
 0.0418812744,
 0.0128296586,
 -0.0468763933,
 -0.0241368003,
 -0.0205882024,
 0.0376288332,
 0.0014298491,
 -0.0611683503,
 0.0250321552,
 -0.0488700345,
 0.0214957725,
 -0.0655383617,
 0.0497455373,
 0.0483916327,
 0.037381243,
 -0.0233520921,
 -0.0035018283,
 0.0493327603,
 0.0558017455,
 -0.0009784676,
 -0.0661733449,
 -0.0507086441,
 -0.0364560001,
 0.005344708,
 -0.0191676207,
 -0.0563165955,
 -0.0657291189,
 -0.021348238,
 -0.0637413561,
 -0.0488439612,
 -0.058114104,
 0.0411729366,
 -0.0649475753,
 -0.0107492562,
 0.0327971205,
 -0.057550054,
 0.0117146475,
 -0.0474368595,
 0.0086091822,
 -0.0650470257,
 -0.01083005

In [240]:
k = 4
D, I = index.search(np.array([query]).astype('float32'),k)
print(I)
print(D)

[[   2 1248  283   79]]
[[0.9999997  0.66516745 0.64820206 0.59296393]]


In [241]:
df.iloc[2][['text','source']]


text      Funds for innovative renewable projects in And...
source                                              ds_pwdb
Name: 8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353, dtype: object

In [242]:
df.iloc[1248][['text','source']]

text      Green energy for accommodation & food business...
source                                              ds_pwdb
Name: c6da8d58564a47f2dc3062a3c480b10a0cb562c803d2c603dd24cefab0adc7c5, dtype: object

In [214]:
emb_list = df[DOCUMENT_EMBEDDING].tolist()
name_list = df.index.to_list()

In [215]:
sim_matrix = pd.DataFrame(pairwise_distances(emb_list, metric=similarity_func),index = name_list, columns = name_list)
sim_matrix

,adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0,2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc,8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353,18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044,b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b,d6897267c1316d4c6be9d6f64332f470e4118225476847bf448abce7ab934145,51fd686c7abd00e1e7b5b31a436a921582b3babe46d1941714b38495ad3fe8ed,c118f163e77cf2c95f981bb2528b821379b5c498eb04ee346babd91d673bf9ec,6e8160a1e50fd0536d4e770490fbcfc20d7e7dc3c65bac762063ac097866947f,6f70ff714553a60e3404d55a941ff47a3e0dd4a9ba5cb9135f3e54f418b3a97f,...,fa25e4c61921862c0310a9802fb459b36b152d51d23ce58f5ec6d0a9251d27c2,fab01840d4d4652594130f3a64de72564dc3a342c516b2560cd7fe29284b82b9,fac3fd4870ae1b04d8c04b750851df65e38b74952c738ca09a5ec3dd9d0eaab1,fb041845214d88926218c03ec39f14e6fb9e46b36d32fa39dab38ce88cae65c3,fb207003cb281a1157be6a5c8580e87d9715c42ad78de4317a0a5b397ef955f3,fb64282022bd7a49e51b2548d2f325a8cbbea756702d619c024cf34c43ec1cfd,fc3912792c86de37ec7f6ffa7a0ce5d5d7b7c86f02ba890cdf0bb85a3628a792,fc8805ee2fd5f28dee01b1172f4ee9d4b3937c8578a92fae6720471685863814,fdd37416fae54311797c08c37956747d741ab86141a64da118c0d3c27a3ca69e,fefa88e8bcb8f19c1794526db7e241a1201a7aa6f05fedd25c09cee4b35c46f0
adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0,1.000000,0.496512,0.453722,0.497151,0.470805,0.533803,0.448236,0.470663,0.454792,0.516943,...,0.411572,0.418112,0.415931,0.420042,0.426732,0.408253,0.421007,0.418130,0.413371,0.415848
2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc,0.496512,1.000000,0.454741,0.496707,0.476957,0.494780,0.458312,0.466210,0.466761,0.488558,...,0.412700,0.424544,0.418685,0.411672,0.427768,0.419538,0.418824,0.417562,0.417337,0.442441
8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353,0.453722,0.454741,1.000000,0.437009,0.469963,0.447736,0.458513,0.466450,0.449098,0.445912,...,0.417419,0.422373,0.431373,0.414413,0.406727,0.407835,0.410753,0.414804,0.457231,0.442620
18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044,0.497151,0.496707,0.437009,1.000000,0.497571,0.515404,0.481000,0.447983,0.456158,0.467616,...,0.413016,0.415274,0.410590,0.435588,0.434046,0.418540,0.418645,0.419202,0.416171,0.435658
b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b,0.470805,0.476957,0.469963,0.497571,1.000000,0.502765,0.501329,0.457158,0.468089,0.460835,...,0.406539,0.413325,0.419462,0.421313,0.409925,0.418367,0.415904,0.400579,0.427719,0.421342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fb64282022bd7a49e51b2548d2f325a8cbbea756702d619c024cf34c43ec1cfd,0.408253,0.419538,0.407835,0.418540,0.418367,0.434927,0.410787,0.425300,0.436270,0.415307,...,0.441992,0.427747,0.434162,0.425351,0.425368,1.000000,0.409047,0.424939,0.427098,0.412986
fc3912792c86de37ec7f6ffa7a0ce5d5d7b7c86f02ba890cdf0bb85a3628a792,0.421007,0.418824,0.410753,0.418645,0.415904,0.421587,0.420348,0.412778,0.412828,0.419839,...,0.411930,0.408673,0.412979,0.403183,0.408329,0.409047,1.000000,0.423722,0.409164,0.414334
fc8805ee2fd5f28dee01b1172f4ee9d4b3937c8578a92fae6720471685863814,0.418130,0.417562,0.414804,0.419202,0.400579,0.413174,0.420232,0.406998,0.402162,0.422787,...,0.420634,0.421194,0.415561,0.417285,0.444605,0.424939,0.423722,1.000000,0.415707,0.415690
fdd37416fae54311797c08c37956747d741ab86141a64da118c0d3c27a3ca69e,0.413371,0.417337,0.457231,0.416171,0.427719,0.417119,0.418934,0.418683,0.420986,0.411670,...,0.406538,0.441341,0.457365,0.415263,0.418713,0.427098,0.409164,0.415707,1.000000,0.445997


In [219]:
sim_matrix_without_diagonal = sim_matrix

In [220]:
np.fill_diagonal(sim_matrix_without_diagonal.values, -1)

In [231]:
sim_matrix_without_diagonal.max(axis=1).sort_values(ascending=False).values[:1000]

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.     

In [234]:
sim_matrix.loc[sim_matrix[sim_matrix.columns[0]] > 0.8]

,adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0,2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc,8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353,18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044,b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b,d6897267c1316d4c6be9d6f64332f470e4118225476847bf448abce7ab934145,51fd686c7abd00e1e7b5b31a436a921582b3babe46d1941714b38495ad3fe8ed,c118f163e77cf2c95f981bb2528b821379b5c498eb04ee346babd91d673bf9ec,6e8160a1e50fd0536d4e770490fbcfc20d7e7dc3c65bac762063ac097866947f,6f70ff714553a60e3404d55a941ff47a3e0dd4a9ba5cb9135f3e54f418b3a97f,...,fa25e4c61921862c0310a9802fb459b36b152d51d23ce58f5ec6d0a9251d27c2,fab01840d4d4652594130f3a64de72564dc3a342c516b2560cd7fe29284b82b9,fac3fd4870ae1b04d8c04b750851df65e38b74952c738ca09a5ec3dd9d0eaab1,fb041845214d88926218c03ec39f14e6fb9e46b36d32fa39dab38ce88cae65c3,fb207003cb281a1157be6a5c8580e87d9715c42ad78de4317a0a5b397ef955f3,fb64282022bd7a49e51b2548d2f325a8cbbea756702d619c024cf34c43ec1cfd,fc3912792c86de37ec7f6ffa7a0ce5d5d7b7c86f02ba890cdf0bb85a3628a792,fc8805ee2fd5f28dee01b1172f4ee9d4b3937c8578a92fae6720471685863814,fdd37416fae54311797c08c37956747d741ab86141a64da118c0d3c27a3ca69e,fefa88e8bcb8f19c1794526db7e241a1201a7aa6f05fedd25c09cee4b35c46f0
